## CVE Analysis Engine

### Global Setup

In [ ]:
import pandas as pd
import numpy as np
import cvss
import cvss.exceptions
import nltk

nltk.download("stopwords")
nltk.download("wordnet")


In [ ]:
df = pd.read_json("../data/cve/cves.json")

df.columns

In [ ]:
def _calc_cvss(v: str) -> float:
    try:
        return cvss.CVSS3(v).scores()[0]
    except cvss.exceptions.CVSS3MalformedError:
        return -1.0


df["scores"] = df["XYZ_CVSS_VECTOR"].dropna().apply(_calc_cvss)
df["bad"] = df["XYZ_CVSS_SCORE"].notna() & (df["XYZ_CVSS_SCORE"] != df["scores"])
df[["XYZ_CVSS_SCORE", "scores", "bad"]].to_csv("../output.csv")


In [ ]:
df.head()

### Attempt 1: Logistic Regression

Logistic Regression is often referred to as the _discriminative_
counterpart of Naive Bayes.

Model $P(y | \mathbf{x}_i)$ and assume it takes exactly the form

$$
    P(y | \mathbf{x}_i) = \frac{1}{1 + e^{-y(\mathbf{w}^T\mathbf{x}_i + b)}}
$$

while making few assumptions about $P(\mathbf{x}_i | y)$.
Ultimately it doesn't matter, because we estimate $\mathbf{w}$ and $b$
directly with MLE or MAP to maximize the conditional likelihood of

$$
    \prod_i P(y_i | \mathbf{x}_i; \mathbf{w}, b)
$$

#### MLE

Choose parameters that maximize the conditional likelihood.
The conditional data likelihood $P(\mathbf{y} | X, \mathbf{w})$
is the probability of the observed values $\mathbf{y} \in \mathbb{R}^n$
in the training data conditioned on the feature values $\mathbf{x}_i$.
Note that $X = [\mathbf{x}_1,\dots,\mathbf{x}_n] \in \mathbb{R}^{d \times n}$.
We choose the parameters that maximize this function, and we assume that
the $y_i$ are independent given the input features $\mathbf{x}_i$ and $\mathbf{w}$.

> In my view, for CVE vectors, this assumption is perfectly valid to make

$$
    P(\mathbf{y} | X, \mathbf{w}) = \prod_{i=1}^{n} P(y_i | \mathbf{x}_i, \mathbf{w}) \\
    \hat{\mathbf{w}}_{\text{MLE}}
    = \underset{\mathbf{w}}{\arg\max}
    - \sum_{i=1}^{n}\log(1 + e^{-y_i\mathbf{w}^T\mathbf{x}_i}) \\
    = \underset{\mathbf{w}}{\arg\min} \sum_{i=1}^{n}\log(1 + e^{-y_i\mathbf{w}^T\mathbf{x}_i})
$$

Use gradient descent on the _negative log likelihood_.

$$
    \ell(\mathbf{w}) = \sum_{i=1}^{n}\log(1 + e^{-y_i\mathbf{w}^T\mathbf{x}_i})
$$

### Text preprocessing

1. lowercase all text
1. remove punctuation
1. tokenize
1. remove stop words
1. lemmatization

In [ ]:
import nltk
import string

def desc_preprocess(d: str):
    # setup
    stopwords = set(nltk.corpus.stopwords.words("english"))
    lemmatizer = nltk.stem.WordNetLemmatizer()

    # lowercase
    d = d.lower()
    # remove punctuation
    d = d.translate(str.maketrans(string.punctuation, " "*len(string.punctuation)))
    # tokenize
    tokens = d.split()
    # remove stop words
    tokens = [t for t in tokens if t not in stopwords]
    tokens = [lemmatizer.lemmatize(t) for t in tokens]
    return tokens


In [ ]:
df["DESCRIPTION"].apply(desc_preprocess).head()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

def create_bow(descs: pd.Series) -> np.ndarray:
    return CountVectorizer().fit_transform(descs).toarray()

X = create_bow(df["DESCRIPTION"])

len(X), len(X.T)



#### Problem statement

There are two sides to my problem:

1. **Given input descriptions, predict the cvss vector.**
   This is a multi-label, multi-class classification problem.
   Some potential strategies are defined below.
1. **Given input descriptions, suggest a cvss score directly.**
   This is probably a regression problem, although it can
   be converted into a classification problem with buckets
   score buckets of some discrete size.

- Independent labels: train a separate classifier for each label, probably using softmax regression
- Dependent labels: classifier chains - input to a classifier includes output from another
- Dependent labels: label powerset - transform problem into a multi-class problem
  with one multi-class classifier is trained on all unique label combinations
  found in the training data.  Deals efficiently with label correlations.

I need to make a decision regarding the independence assumption of my labels.
I find it intellectually interesting to explore the correlation statistics
between the category + label combinations.  Two methods for establishing
correlation between categories is
- Chi-square test of independence
- Cramer's V

#### Next steps

- Look at documentation to make sure I've got my problem statements right.
  Does vector suggestion deliver value?
- Perform a *Cramer's V* analysis on training examples
- Based on the output of this, decide on ml strategy


In [ ]:
from typing import Union

metrics = {
    "AV": "Attack Vector",
    "AC": "Attack Complexity",
    "PR": "Privileges Required",
    "UI": "User Interaction",
    "S": "Scope",
    "C": "Confidentiality",
    "I": "Integrity",
    "A": "Availability",
}


def _vec_parse(vec: str, metric: str):
    return cvss.CVSS3(vec).get_value_description(metric)

def clean_cvss_vector(vec: Union[str, float]) -> Union[str, float]:
    if pd.isna(vec): return np.nan
    try:
        return cvss.CVSS3(vec).clean_vector()
    except cvss.exceptions.CVSS3MalformedError:
        pass

    # fix common problems
    assert type(vec) is str
    vec = vec.upper()
    vec = vec.rstrip(".")
    vec = vec.replace(" ", "")
    vec = vec.rstrip("/")
    try:
        vec = "CVSS:3.1/" + vec[vec.index("AV:"):]
    except ValueError:
        pass
    # vec = vec.removeprefix("VECTOR:")
    # if vec.startswith("AV"): vec = "CVSS:3.1/" + vec
    # if vec.startswith("/AV"): vec = "CVSS:3.1" + vec

    # try again
    try:
        return cvss.CVSS3(vec).clean_vector()
    except cvss.exceptions.CVSS3MalformedError:
        return np.nan
    

def extract_cvss_vector_components(df: pd.DataFrame, vector: pd.Series):
    for metric in metrics.keys():
        df[metric] = vector.dropna().apply(lambda v: _vec_parse(v, metric))
    return df


In [ ]:
df["vector"] = df.XYZ_CVSS_VECTOR.apply(clean_cvss_vector)

In [ ]:
df = extract_cvss_vector_components(df, df["vector"])
print(df["vector"].count())

df.to_csv("../asdf.csv")

In [ ]:
[df[metric].count() for metric in metrics.keys()]

Pearson's $\Chi^2$ test

- [Pearson's chi-squared test - Wikipedia](https://en.wikipedia.org/wiki/Pearson%27s_chi-squared_test)

Based on the below analysis, a couple of the highest correlations:

- AV:L & PR:L highly correlated
- AV:N & PR:L highly negatively correlated
- PR:L & UI:R highly negatively correlated
- PR:N & UI:R highly correlated
- PR:H & S:C highly correlated
- I:L & S:C very highly correlated
- A:N & S:C very highly correlated
- C:* & I* extemely correlated both positively and negatively

In [ ]:
import statsmodels.api as sm
import itertools

crosstabs = {}

def perform_independence_test(df: pd.DataFrame):
    for c0, c1 in itertools.combinations(metrics.keys(), 2):
        xtab = pd.crosstab(df[c0], df[c1])
        crosstabs[":".join((c0,c1))] = xtab
        print(f"\n=== {c0} & {c1}")
        print(sm.stats.Table(xtab).resid_pearson)

perform_independence_test(df)

### Testing for statistical significance of cross-category dependence

$H_{0_{\alpha, \beta}}$: metric $\alpha$ and metric $\beta$ are independent

Use standard significance level $\alpha = 0.5$.

In [ ]:
import scipy

xtab = crosstabs["C:I"]
alpha = 0.5
chi2stat, pvalue, dof, expected_frequency = scipy.stats.chi2_contingency(xtab)
chi2stat, pvalue, dof, expected_frequency, pvalue <= alpha

### Understanding CVSS Vectors

The following table lists out the metrics comprised by the CVSS vector.
More info can be found in the [CVSS v3.1 Specification Document](https://www.first.org/cvss/specification-document).
Each metric contributes a predefined amount to the overall CVE CVSS score,
and the score is completely determined by the values of these metrics.
Values in the table below are ordered from most to least severe.

| **Base metric** | **Base metric type** | **Description** | **Possible values** |
|---|---|---|---|
| Attack Vector (AV) | Exploitability | This metric reflects the context by which vulnerability exploitation is possible. | Network, Adjacent, Local, Physical |
| Attack Complexity (AC) | Exploitability | This metric describes the conditions beyond the attacker’s control that must exist in order to exploit the vulnerability. | Low, High |
| Privileges Required (PR) | Exploitability | This metric describes the level of privileges an attacker must possess before successfully exploiting the vulnerability. | None, Low, High |
| User Interaction (UI) | Exploitability | This metric captures the requirement for a human user, other than the attacker, to participate in the successful compromise of the vulnerable component. | None, Required |
| Scope (S) | Scope¹ | The Scope metric captures whether a vulnerability in one vulnerable component impacts resources in components beyond its security scope. | Changed, Unchanged |
| Confidentiality (C) | Impact | This metric measures the impact to the confidentiality of the information resources managed by a software component due to a successfully exploited vulnerability. | High, Low, None |
| Integrity (I) | Impact | This metric measures the impact to integrity of a successfully exploited vulnerability. | High, Low, None |
| Availability (A) | Impact | This metric measures the impact to the availability of the impacted component resulting from a successfully exploited vulnerability. | High, Low, None |

<br>

> ¹Scope was introduced in CVSS3.1.

For our model, the precise meaning of these metrics and their subcategories is unimportant.
The relevant question is: _how independent are these metrics and subcategories?_

Assumption of label independence certainly makes our job easier,
as it leaves the door open for more basic machine learning algorithms
like **multi-category logistic regression**.

In [ ]:
import numpy as np
import scipy.stats

# Generate two sets of data
data1 = np.random.normal(0, 1, 1000)
data2 = np.random.normal(0.1, 1, 1000)

# Perform a t-test
t_stat, p_value = scipy.stats.ttest_ind(data1, data2)

print(f"t-statistic: {t_stat}")
print(f"p-value: {p_value}")


Need to understand if the results in the crosstable are statistically significant
Then, select an approach for training (multi-class log reg?)

Then, try a basic run